In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import os
import numpy as np
import shutil
import re
import pandas as pd
import urllib.request


In [2]:
text = []
links = []
dirs = "./html"

if os.path.exists(dirs):
    shutil.rmtree(dirs)
os.makedirs(dirs)
for year in range(2012, 2021):
    
    url = "https://labor.ny.gov/app/warn/default.asp?warnYr="+str(year)
    print(url)
    page = requests.get(url)
    page.encoding = 'ISO-8859-1'
    soup = BeautifulSoup(page.text, 'html.parser')
    table_data = soup.findAll('td')
    table_data = [ele for ele in table_data if re.sub(' +', ' ', ele.text).strip()]
    base_url = "https://labor.ny.gov/app/warn/"
    sub_url = [base_url+ele.find('a').get('href') for ele in table_data]



    for ele in sub_url:
        opener = urllib.request.build_opener()
        opener.addheaders = [('User-agent', 'Mozilla/5.0')]
        page = opener.open(ele).read().decode('utf-8')


        with open(os.path.join(dirs, "NY_"+str(year)+"_"+ele.split('=')[1]+".html"), 'w',encoding = 'utf8') as file:
            file.write(page)
        soup = BeautifulSoup(page, 'html.parser')

        table_data = soup.find('td')
        content = table_data.text

        elements = [ele.replace(u'\xa0', u'') for ele in re.split("[\n]",content)]
        elements = [ele for ele in elements if ele]

        links.append(ele)
        text.append(elements)
        #print(elements)

https://labor.ny.gov/app/warn/default.asp?warnYr=2012
https://labor.ny.gov/app/warn/default.asp?warnYr=2013
https://labor.ny.gov/app/warn/default.asp?warnYr=2014
https://labor.ny.gov/app/warn/default.asp?warnYr=2015
https://labor.ny.gov/app/warn/default.asp?warnYr=2016
https://labor.ny.gov/app/warn/default.asp?warnYr=2017
https://labor.ny.gov/app/warn/default.asp?warnYr=2018
https://labor.ny.gov/app/warn/default.asp?warnYr=2019
https://labor.ny.gov/app/warn/default.asp?warnYr=2020


In [4]:
text2 = []
for ele in text:
    result = {}
    symbol = "&&"
    string = symbol.join(ele)
    string = re.sub(' +', ' ', string).strip()
    string = string.replace("Event Number","Control Number",1)
    #print(string)
    #print(string)
    titles = ['Date of Notice', 'Control Number', 'Rapid Response Specialist', 'Reason Stated for Filing', 'Company', 'County', 'Contact', 'Phone', 'Business Type', 'Number Affected', 'Total Employees', 'Layoff Date', 'Closing Date', 'Reason for Dislocation', 'ERNUM', 'Union', 'Classification', symbol]
    for i in range(len(titles)-1):
        try:
            tt = string.split(titles[i])[1].split(titles[i+1])[0].replace(":", "", 1).replace(symbol, "", 1)
            result[titles[i]] = re.sub(' +', ' ', tt).strip()
        except:
            result[titles[i]] = ""
    try:
        tt_note = string.split(titles[-2])[1].split(symbol,1)[1].replace(symbol, "", 1)
        result['notes'] = re.sub(' +', ' ', tt_note).strip()
    except:
        result['notes'] = ""
    text2.append(result)

In [ ]:
#result

In [5]:
format_error = {}
titles = ['Date of Notice', 'Control Number', 'Rapid Response Specialist', 'Reason Stated for Filing', 'Company', 'County', 'Contact', 'Phone', 'Business Type', 'Number Affected', 'Total Employees', 'Layoff Date', 'Closing Date', 'Reason for Dislocation', 'ERNUM', 'Union', 'Classification', 'notes']
for title in titles:
    format_error[title] = [ele[title] for ele in text2]
format_error['link'] = links

In [6]:
df_format = pd.DataFrame(data = format_error)
df_final = df_format[(df_format['Control Number'] != '') & (df_format['Date of Notice'] != '') & (df_format['Rapid Response Specialist'] != '')]
df_final.to_csv("final_result.csv", index = False)

In [7]:
df_error = df_format[(df_format['Control Number'] == '') | (df_format['Date of Notice'] == '') | (df_format['Rapid Response Specialist'] == '')]

In [8]:
df_error.to_csv("final_error.csv", index = False)

In [ ]:
#Reference:
#https://regexr.com/